### Постановка задачи
Посчитаем модель линейной регрессии по первым 100 зданиям и найдем ее точность, используя в качестве параметров только дни недели и праздники, применяя fit_intercept=False и логарифмируя целевой показатель.

Для вычисления отсутствующих или некорректных данных построим модели по всем зданиям одного типа в одном городе и во всех городах.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Загрузка данных, отсечение 100 зданий, объединение и оптимизация

In [2]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[(energy["building_id"]<100)]
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "year_built",
                              "square_feet", "floor_count"], axis=1)
del buildings
del weather
energy = reduce_mem_usage(energy)
print (energy.info())

Потребление памяти меньше на 56.89 Мб (минус 71.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864557 entries, 0 to 864556
Data columns (total 12 columns):
timestamp             864557 non-null datetime64[ns]
site_id               864557 non-null int8
building_id           864557 non-null int8
meter_reading         864557 non-null float16
primary_use           864557 non-null category
air_temperature       864263 non-null float16
cloud_coverage        487693 non-null float16
dew_temperature       864263 non-null float16
precip_depth_1_hr     864459 non-null float16
sea_level_pressure    856210 non-null float16
wind_direction        839970 non-null float16
wind_speed            864557 non-null float16
dtypes: category(1), datetime64[ns](1), float16(8), int8(2)
memory usage: 22.3 MB
None


### Обогащение данных: час, дни недели, праздники, логарифм

In [11]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

### Разделение данных

In [13]:
energy_train, energy_test = train_test_split(energy[(energy["meter_reading"]>0)], test_size=0.2)
print (energy_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427872 entries, 860794 to 336212
Data columns (total 24 columns):
timestamp             427872 non-null datetime64[ns]
site_id               427872 non-null int8
building_id           427872 non-null int8
meter_reading         427872 non-null float16
primary_use           427872 non-null category
air_temperature       427867 non-null float16
cloud_coverage        248978 non-null float16
dew_temperature       427867 non-null float16
precip_depth_1_hr     427870 non-null float16
sea_level_pressure    425639 non-null float16
wind_direction        414132 non-null float16
wind_speed            427872 non-null float16
hour                  427872 non-null int8
weekday               427872 non-null int8
is_wday0              427872 non-null int8
is_wday1              427872 non-null int8
is_wday2              427872 non-null int8
is_wday3              427872 non-null int8
is_wday4              427872 non-null int8
is_wday5              427872 

### Линейная регрессия: по часам

In [15]:
hours = range(0, 24)
buildings = range(0, energy_train["building_id"].max() + 1)
lr_columns = ["meter_reading_log", "hour", "building_id", "is_holiday"]
for wday in range(0,7):
    lr_columns.append("is_wday" + str(wday))
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns)
energy_lr = [[]]*len(buildings)
for building in buildings:
    energy_lr[building] = [[]]*len(hours)
    energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
    for hour in hours:
        energy_lr[building].append([0]*(len(lr_columns)-3))
        energy_train_bh = pd.DataFrame(energy_train_b[energy_train_b["hour"]==hour])
        y = energy_train_bh["meter_reading_log"]
        if len(y) > 0:
            x = energy_train_bh.drop(labels=["meter_reading_log",
                "hour", "building_id"], axis=1)
            model = LinearRegression(fit_intercept=False).fit(x, y)
            energy_lr[building][hour] = model.coef_
            energy_lr[building][hour] = np.append(energy_lr[building][hour], model.intercept_)
print (energy_lr[0])

[array([-0.07672264,  5.44733172,  5.48182745,  5.47721354,  5.42741865,
        5.48529313,  5.48270089,  5.48386549,  0.        ]), array([-0.15666387,  5.4587836 ,  5.45023148,  5.45401278,  5.48535156,
        5.45342968,  5.47435462,  5.47386853,  0.        ]), array([-0.11096011,  5.43725553,  5.42357337,  5.4938151 ,  5.46340231,
        5.46114871,  5.44015625,  5.42169744,  0.        ]), array([-0.15330244,  5.45772624,  5.46831597,  5.49652778,  5.46149029,
        5.4555444 ,  5.48141164,  5.44215745,  0.        ]), array([-0.13053489,  5.46305359,  5.41878255,  5.48370151,  5.44810116,
        5.45115642,  5.44285301,  5.42983774,  0.        ]), array([-0.13158166,  5.46870578,  5.45918642,  5.47578125,  5.45368304,
        5.4471477 ,  5.43457031,  5.42787388,  0.        ]), array([-0.18098801,  5.5090577 ,  5.40987723,  5.46679688,  5.45813295,
        5.45484555,  5.41682943,  5.45173891,  0.        ]), array([-0.11250546,  5.49010508,  5.47352431,  5.49881114,  5.496066

### Линейная регрессия: по типам зданий

In [16]:
sites = range(0, energy["site_id"].max() + 1)
primary_uses = energy["primary_use"].unique()
lr_columns_use = ["meter_reading_log", "hour", "building_id",
                  "is_holiday", "primary_use", "site_id"]
for wday in range(0,7):
    lr_columns_use.append("is_wday" + str(wday))
energy_lr_use = {}
energy_lr_use_site = {}
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns_use)
for primary_use in primary_uses:
    energy_train_u = energy_train_lr[energy_train_lr["primary_use"]==primary_use]
    if len(energy_train_u) > 0:
        energy_lr_use_site[primary_use] = [[]]*len(sites)
        for site in sites:
            energy_lr_use_site[primary_use][site] = [[]]*len(hours)
            energy_train_us = energy_train_u[energy_train_u["site_id"]==site]
            if len(energy_train_us) > 0:
                for hour in hours:
                    energy_train_uth = energy_train_us[energy_train_us["hour"]==hour]
                    y = energy_train_uth["meter_reading_log"]
                    if len(y) > 0:
                        x = energy_train_uth.drop(labels=["meter_reading_log",
                            "hour", "building_id", "site_id",
                            "primary_use"], axis=1)
                        model = LinearRegression(fit_intercept=False).fit(x, y)
                        energy_lr_use_site[primary_use][site][hour] = model.coef_
                        energy_lr_use_site[primary_use][site][hour] = np.append(energy_lr_use_site[primary_use][site][hour], model.intercept_)
        energy_lr_use[primary_use] = [[]]*len(hours)
        for hour in hours:
            energy_train_th = energy_train_u[energy_train_u["hour"]==hour]
            y = energy_train_th["meter_reading_log"]
            if len(y) > 0:
                x = energy_train_th.drop(labels=["meter_reading_log",
                    "hour", "building_id", "site_id", "primary_use"], axis=1)
                model = LinearRegression(fit_intercept=False).fit(x, y)
                energy_lr_use[primary_use][hour] = model.coef_
                energy_lr_use[primary_use][hour] = np.append(energy_lr_use[primary_use][hour], model.intercept_)
print (energy_lr_use_site)

{'Education': [[array([-4.95108910e-03,  5.61360437e+00,  5.65882530e+00,  5.67235354e+00,
        5.68624227e+00,  5.66868724e+00,  5.66855781e+00,  5.61365798e+00,
        0.00000000e+00]), array([-0.09751058,  5.57509723,  5.63411768,  5.64540389,  5.67529104,
        5.64540242,  5.62053826,  5.5898385 ,  0.        ]), array([-0.09047305,  5.58844298,  5.59584691,  5.64209836,  5.62227457,
        5.64149599,  5.58717135,  5.55590392,  0.        ]), array([-0.07769471,  5.6086802 ,  5.63253348,  5.61769038,  5.64547642,
        5.6607634 ,  5.61264597,  5.58146314,  0.        ]), array([-0.11248114,  5.62500199,  5.6085948 ,  5.63014563,  5.65697546,
        5.63792365,  5.58827664,  5.56795446,  0.        ]), array([-0.00847801,  5.60303734,  5.6189954 ,  5.61888264,  5.67267042,
        5.62066229,  5.64663652,  5.5692787 ,  0.        ]), array([-0.05374606,  5.73390029,  5.71455242,  5.70093493,  5.70687447,
        5.71191509,  5.68832156,  5.7302437 ,  0.        ]), array([-0.

### Расчет качества
Используем индивидуальные модели здания, иначе общую модель по всем зданиям данного типа в городе, иначе общую модель по всем зданиям такого типа (по всем городам)

In [17]:
def calculate_model (x):
    lr = -1
    model = energy_lr[x.building_id][x.hour]
    if len(model) == 0:
        model = energy_lr_use_site[x.primary_use][x.site_id][x.hour]
    if len(model) == 0:
        model = energy_lr_use[x.primary_use][x.hour]
    if len(model) > 0:
        lr = np.sum([x[col] * model[i] for i,col in enumerate(lr_columns[3:])])
        lr += model[len(lr_columns)-3]
        lr = np.exp(lr)
    if lr < 0:
        lr = 0
    x["meter_reading_lr_q"] = (np.log(x.meter_reading + 1) -
                               np.log(1 + lr))**2
    return x

energy_test = energy_test.apply(calculate_model,
                                    axis=1, result_type="expand")
energy_test_lr_rmsle = np.sqrt(energy_test["meter_reading_lr_q"].sum() / len(energy_test))
print ("Качество линейной регрессии, 100 зданий:",
       energy_test_lr_rmsle, round(energy_test_lr_rmsle, 1))

Качество линейной регрессии, 100 зданий: 0.33775986225141585 0.3
